In [1]:
# Импорт библиотек
import sqlite3 
from sqlite3 import Error 
import pandas as pd

**Шаг 1.** Подготовительный этап  
*Аналогичные действия вы делали в прошлом домашнем задании.*  

(Балл - 2) Необходимо скачать CSV-файл - «transactions_for_dz2.csv», создать таблицу transaction_bd со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:  

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции  
(1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции  
(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента  
(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала  
(4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции  

Также необходимо скачать второй CSV-файл - «client_info.csv», создать таблицу customer_bd со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:  

(0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента  
(1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте  
(2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте  
(3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента  
(4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождения клиента  

In [2]:
# Функция создания Connection с БД
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

In [3]:
# Создание Connection с БД. Неявно создает БД, если не существует.
con = create_connection("HW4.db")

Connection to SQLite DB successful


In [4]:
# Функция выполнения запроса к БД. Создание Cursor, выполнение запроса/ошибка, если запрос выполнен, то 
# коммит Connection, закрытие Cursor.
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
        cursor.close()
    except Error as e:
        print(f"The error '{e}' occurred")

In [5]:
# Запрос на создание таблицы transaction_bd в БД.
create_transaction_table = """
CREATE TABLE IF NOT EXISTS transaction_bd (
  TRANSACTION_ID INTEGER NOT NULL,
  TX_DATETIME TEXT NOT NULL,
  CUSTOMER_ID INTEGER NOT NULL,
  TERMINAL_ID INTEGER NOT NULL,
  TX_AMOUNT REAL NOT NULL
);
"""

In [6]:
# Исполнение запроса на создание таблицы transaction_bd в БД.
execute_query(con, create_transaction_table)

Query executed successfully


In [7]:
# Запрос на создание таблицы customer_bd в БД.
create_customer_table = """
CREATE TABLE IF NOT EXISTS customer_bd (
  CLIENT_ID INTEGER NOT NULL,
  START_DT TEXT NOT NULL,
  END_DT TEXT NOT NULL,
  CLIENT_NAME TEXT NOT NULL,
  YEAR_BIRTH TEXT NOT NULL
);
"""

In [8]:
# Исполнение запроса на создание таблицы customer_bd в БД.
execute_query(con, create_customer_table)

Query executed successfully


In [9]:
# Чтение файла csv в датафрейм.
df_trans = pd.read_csv('transactions_for_dz2.csv')
df_trans

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66
...,...,...,...,...,...
1048570,1048570,2023-04-20 10:07:13,2380,3780,325.64
1048571,1048571,2023-04-20 10:07:28,738,5151,20.38
1048572,1048572,2023-04-20 10:07:33,1000,5417,182.79
1048573,1048573,2023-04-20 10:07:39,3028,6439,455.44


In [10]:
# Кол-во уникальных клиентов в датафрейме/таблице.
df_trans['CUSTOMER_ID'].nunique()

4986

In [11]:
# Добавление всех записей с файла transactions_for_dz2.csv в таблицу transaction_bd
df_trans.to_sql('transaction_bd', con, if_exists='append', index=False)

In [12]:
# Чтение файла csv в датафрейм.
df_cust = pd.read_csv('client_info.csv', sep=';')
df_cust

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148
2,2015-07-18,2021-01-01,Charlotte,1979,2293
3,2015-07-19,2021-01-01,Amelia,1979,1867
4,2015-07-20,2021-01-01,Ava,1979,1767
...,...,...,...,...,...
4983,2023-04-20,2999-12-31,Havilah,2000,4737
4984,2023-04-20,2999-12-31,Hazelyn,2000,2301
4985,2023-04-20,2999-12-31,Helaina,2000,3238
4986,2023-04-20,2999-12-31,Helene,2000,3946


In [13]:
# Информация о действующих клиентах.
df_cust_active = df_cust[df_cust['END_DT'] >= '2023-05-01']
df_cust_active

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
27,2010-01-01,2999-12-31,Hazel,1945,1172
28,2010-01-01,2999-12-31,Madison,1945,2912
29,2010-01-01,2999-12-31,Ellie,1945,3604
30,2010-01-01,2999-12-31,Lily,1946,4361
31,2010-02-01,2999-12-31,Nova,1946,3700
...,...,...,...,...,...
4983,2023-04-20,2999-12-31,Havilah,2000,4737
4984,2023-04-20,2999-12-31,Hazelyn,2000,2301
4985,2023-04-20,2999-12-31,Helaina,2000,3238
4986,2023-04-20,2999-12-31,Helene,2000,3946


In [14]:
# Транзакции действующих клиентов.
trans_active = df_trans[df_trans['CUSTOMER_ID'].isin(set(df_cust_active['CLIENT_ID']))]
trans_active

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
5,5,2023-01-01 00:10:45,568,8803,401.17
6,6,2023-01-01 00:11:30,2803,5490,938.54
...,...,...,...,...,...
1048568,1048568,2023-04-20 10:07:01,3670,8244,616.24
1048569,1048569,2023-04-20 10:07:05,1534,4927,80.99
1048570,1048570,2023-04-20 10:07:13,2380,3780,325.64
1048571,1048571,2023-04-20 10:07:28,738,5151,20.38


In [15]:
# ID действующих клиентов, которые не совершали транзакции за период анализа.
set(df_cust_active['CLIENT_ID']) - set(trans_active['CUSTOMER_ID'])

{11111, 77777}

In [16]:
# Информация о действующих клиентах, которые не совершали транзакции за период анализа.
df_cust_active[df_cust_active['CLIENT_ID'].isin(['11111', '77777'])]

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
4918,2023-03-01,2999-12-31,Sofia,1999,77777
4919,2021-11-01,2999-12-31,Oksi,2000,11111


In [17]:
# Добавление всех записей с файла client_info.csv в таблицу customer_bd
df_cust.to_sql('customer_bd', con, if_exists='append', index=False)

**Шаг 2**  
(Балл - 2 за каждый пункт) Написать скрипты:  

**2a.** Вывести список всех клиентов, у которых между двумя ближайшими транзакциями был перерыв больше 35 дней хотя бы один раз (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [18]:
# Запрос для пункта 2a
query_a = """
with part_lag as(
SELECT CUSTOMER_ID, CLIENT_NAME, TX_DATETIME,
        LAG(TX_DATETIME) OVER(PARTITION BY CUSTOMER_ID ORDER BY TX_DATETIME
        ) AS 'Lag' 
FROM transaction_bd
LEFT JOIN customer_bd ON CUSTOMER_ID = CLIENT_ID
WHERE END_DT >= '2023-05-01'
)
SELECT DISTINCT CUSTOMER_ID, CLIENT_NAME
FROM part_lag
WHERE julianday(TX_DATETIME) - julianday(Lag) > 35
;
"""

In [19]:
# Выполнение запроса для пункта 2a и запись в датафрейм.
df_a = pd.read_sql_query(query_a, con)
df_a

,CUSTOMER_ID,CLIENT_NAME
0,24,Kiarra
1,480,Vaishnavi
2,639,Malayna
3,707,Becca
4,812,Avionna
5,896,Macarena
6,1129,Kavya
7,1299,Nalah
8,1334,Blakley
9,1459,Nalia


**2b.** Вывести список клиентов, у которых максимальная сумма транзакции (имеется ввиду максимальное значение TX_AMOUNT) больше 50000. В ответе должен быть id клиента, максимальное значение суммы транзакций, минимальное значение суммы транзакций (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [20]:
# Запрос для пункта 2b
query_b = """
SELECT CUSTOMER_ID, MAX(TX_AMOUNT) AS MAX_AMOUNT, MIN(TX_AMOUNT) AS MIN_AMOUNT
FROM transaction_bd
LEFT JOIN customer_bd ON CUSTOMER_ID = CLIENT_ID
WHERE END_DT >= '2023-05-01'
GROUP BY CUSTOMER_ID
HAVING MAX(TX_AMOUNT) > 50000
;
"""

In [21]:
# Выполнение запроса для пункта 2b и запись в датафрейм.
df_b = pd.read_sql_query(query_b, con)
df_b

,CUSTOMER_ID,MAX_AMOUNT,MIN_AMOUNT
0,3013,77212.50,1.46
1,3494,53213.00,85.76
2,4253,51937.25,37.95


**2c.** Вывести топ-10 клиентов (сортируя их по убыванию id), у которых сумма дневных транзакций больше суммы вечерних транзакций Условимся, что дневными считаются транзакции, которые были произведены до 12 часов, а вечерними - после 12 часов включительно. (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [22]:
# Запрос для пункта 2c
query_c = """
with day_night as
(SELECT CUSTOMER_ID,
        TX_DATETIME,
        CASE 
        WHEN strftime('%H:%M:%S', TX_DATETIME) BETWEEN '00:00:00' AND '11:59:59' THEN 'day'
        ELSE 'night'
        END AS part_day,
        TX_AMOUNT
FROM transaction_bd
LEFT JOIN customer_bd ON CUSTOMER_ID = CLIENT_ID
WHERE END_DT >= '2023-05-01'
),
diff_table as
(SELECT CUSTOMER_ID,
        SUM(TX_AMOUNT) FILTER (WHERE part_day = 'day') OVER (PARTITION BY CUSTOMER_ID) - 
        SUM(TX_AMOUNT) FILTER (WHERE part_day = 'night') OVER (PARTITION BY CUSTOMER_ID) AS diff_amount
FROM day_night
)
SELECT DISTINCT CUSTOMER_ID
FROM diff_table
WHERE diff_amount > 0
ORDER BY CUSTOMER_ID DESC
LIMIT 10
;
"""

In [23]:
# Выполнение запроса для пункта 2c и запись в датафрейм.
df_c = pd.read_sql_query(query_c, con)
df_c

,CUSTOMER_ID
0,4997
1,4993
2,4990
3,4987
4,4983
5,4982
6,4978
7,4976
8,4973
9,4972


**2d.** Найти для каждого клиента день, когда у него была максимальная сумма транзакции, то есть вывести клиента, дату транзакции, сумму транзакции (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [24]:
# Запрос для пункта 2d
query_d = """
with max_date as
(SELECT CUSTOMER_ID, strftime('%Y-%m-%d', TX_DATETIME) AS DATE, TX_AMOUNT,
        MAX(TX_AMOUNT) OVER(PARTITION BY CUSTOMER_ID) AS MAX_AMOUNT
FROM transaction_bd
LEFT JOIN customer_bd ON CUSTOMER_ID = CLIENT_ID
WHERE END_DT >= '2023-05-01'
)
SELECT CUSTOMER_ID, DATE, MAX_AMOUNT
FROM max_date
WHERE MAX_AMOUNT = TX_AMOUNT
"""

In [25]:
# Выполнение запроса для пункта 2d и запись в датафрейм.
df_d = pd.read_sql_query(query_d, con)
df_d

,CUSTOMER_ID,DATE,MAX_AMOUNT
0,0,2023-02-01,1258.86
1,1,2023-02-07,17152.00
2,2,2023-02-21,1720.11
3,5,2023-03-21,1156.11
4,6,2023-03-20,369.26
...,...,...,...
3731,4991,2023-01-25,1070.26
3732,4992,2023-03-10,839.03
3733,4993,2023-01-13,1054.57
3734,4996,2023-01-11,224.62


In [26]:
# Информация о действующих клиентах, которые не совершали транзакции за период анализа.
df_cust_active[df_cust_active['CLIENT_ID'].isin(['11111', '77777'])]

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
4918,2023-03-01,2999-12-31,Sofia,1999,77777
4919,2021-11-01,2999-12-31,Oksi,2000,11111


In [27]:
# Запрос для пункта 2d
query_d1 = """
SELECT CUSTOMER_ID, strftime('%Y-%m-%d', TX_DATETIME) AS DATE, MAX(TX_AMOUNT) AS MAX_AMOUNT
FROM transaction_bd
LEFT JOIN customer_bd ON CUSTOMER_ID = CLIENT_ID
WHERE END_DT >= '2023-05-01'
GROUP BY CUSTOMER_ID
"""

In [28]:
# Выполнение запроса для пункта 2d и запись в датафрейм.
df_d1 = pd.read_sql_query(query_d1, con)
df_d1

,CUSTOMER_ID,DATE,MAX_AMOUNT
0,0,2023-02-01,1258.86
1,1,2023-02-07,17152.00
2,2,2023-02-21,1720.11
3,5,2023-03-21,1156.11
4,6,2023-03-20,369.26
...,...,...,...
3731,4991,2023-01-25,1070.26
3732,4992,2023-03-10,839.03
3733,4993,2023-01-13,1054.57
3734,4996,2023-01-11,224.62


**2e.** Вычислить интегральное распределение суммы транзакций для каждого дня за весь период, то есть вывести дату транзакции, сумму транзакции и интегральное распределение по сумме (не учитываем что клиент может быть недействующим)

In [29]:
# Запрос для пункта 2e
query_e = """
SELECT DATE, SUM_AMOUNT,
        CUME_DIST() OVER(ORDER BY SUM_AMOUNT) AS 'Cume_Dist'
FROM 
(
SELECT strftime('%Y-%m-%d', TX_DATETIME) AS DATE,
        SUM(TX_AMOUNT) AS SUM_AMOUNT
FROM transaction_bd
GROUP BY strftime('%Y-%m-%d', TX_DATETIME)
)
;
"""

In [30]:
# Выполнение запроса для пункта 2e и запись в датафрейм.
df_e = pd.read_sql_query(query_e, con)
df_e

,DATE,SUM_AMOUNT,Cume_Dist
0,2023-04-20,1795235.72,0.009091
1,2023-01-01,4827656.26,0.018182
2,2023-01-02,4862551.41,0.027273
3,2023-01-04,4938142.47,0.036364
4,2023-02-21,4968545.25,0.045455
...,...,...,...
105,2023-02-09,5364510.25,0.963636
106,2023-04-01,5383710.07,0.972727
107,2023-04-06,5429039.85,0.981818
108,2023-01-23,5509342.65,0.990909


In [31]:
# Запрос для пункта 2e
query_e1 = """
SELECT strftime('%Y-%m-%d', TX_DATETIME) AS DATE,
        TX_AMOUNT,
        CUME_DIST() OVER(PARTITION BY strftime('%Y-%m-%d', TX_DATETIME) ORDER BY TX_AMOUNT) AS 'Cume_Dist'
FROM transaction_bd
;
"""

In [32]:
# Выполнение запроса для пункта 2e и запись в датафрейм.
df_e1 = pd.read_sql_query(query_e1, con)
df_e1
# Интегральное распределение внутри дня для каждого дня за весь период.

,DATE,TX_AMOUNT,Cume_Dist
0,2023-01-01,0.81,0.000105
1,2023-01-01,1.02,0.000211
2,2023-01-01,1.03,0.000316
3,2023-01-01,1.08,0.000422
4,2023-01-01,1.36,0.000527
...,...,...,...
1048570,2023-04-20,13024.50,0.998819
1048571,2023-04-20,18554.25,0.999114
1048572,2023-04-20,19174.75,0.999409
1048573,2023-04-20,19740.75,0.999705


In [33]:
# Закрытие Connection с БД.
con.close()